In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

## Load Data

In [2]:
# change to: "normalized" => "min_max" and "standardized" => "zscore"
normalization = "normalized" 

In [3]:
###
#  load data: targets and features
###
file = h5py.File("targets_and_{}_features.jld2".format(normalization), "r")

# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

In [4]:
###
#  load data: initializing COFs
###
init_cof_ids_file = pickle.load(
                    open('search_results/{}/initializing_cof_ids_{}.pkl'.format(normalization, normalization), 
                         'rb'))

init_cof_ids = init_cof_ids_file['init_cof_ids']

# total number of BO searches to run = number of initializing sets
nb_runs = len(init_cof_ids)

###
#  print info
###
print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
width:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)


In [5]:
X_unsqueezed = X.unsqueeze(1)

## Helper Functions

#### Analysis

In [6]:
# accumulated cost lags the index of the cost_acquired (iteration COF is identified) by 1
def accumulated_cost(cost_acquired):
    nb_iters = len(cost_acquired)
    accumulated_cost = np.zeros(nb_iters)
    accumulated_cost[0] = cost_acquired[0]
    for i in range(1, len(cost_acquired)):
        accumulated_cost[i] = accumulated_cost[i-1] + cost_acquired[i]
    return accumulated_cost

In [7]:
def get_y_maxes_acquired(y_acquired):
    nb_iters = len(y_acquired)
    return [max(y_acquired[:i+1]) for i in range(nb_iters)]      

#### Construct Initial Inputs

In [8]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [9]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1))
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [10]:
def run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=False):
    assert nb_iterations > len(initializing_COFs)
    ###
    #  initialize system
    ###
    # select initial COFs for training data
    ids_acquired = initializing_COFs
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(len(initializing_COFs), nb_iterations):
        # construct training data (perform experiments)
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # identify COF with highest acquisition value currently *not* in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        if verbose:
            print("BO iteration ", i)
            print("\tacquired COF ", id_max_acq)
            print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [11]:
nb_iterations = 150
nb_COFs_initialization = 3

In [12]:
for i, initializing_COFs in enumerate(init_cof_ids): 
    # check the length of each initializing set
    assert len(initializing_COFs) == nb_COFs_initialization
    print("run #: {}".format(i))
    
    # start timer for BO run
    start_time    = time.time()
    ###
    #  run BO search
    ###
    ids_acquired  = run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=True)
    
    ###
    #  post-run analysis
    ###
    elapsed_time  = time.time() - start_time
    y_acquired    = y[ids_acquired]
    y_maxes_acq   = get_y_maxes_acquired(y_acquired.detach().numpy())
    cost_acquired = build_cost(ids_acquired)
    acc_cost      = accumulated_cost(cost_acquired.detach().numpy())
        
    BO_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
    print("iteration we acquire top COF = ", BO_iter_top_cof_acquired.item())
    
    top_cof_acc_cost = sum(cost_acquired[:BO_iter_top_cof_acquired]).item() 
    print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]")
    
    ###
    #  Store SFBO results
    ###
    sfbo_res = dict({'ids_acquired': ids_acquired,
                     'y_acquired': y_acquired.detach().numpy(),
                     'y_max_acquired': y_maxes_acq,
                     'cost_acquired': cost_acquired.detach().numpy(),
                     'accumulated_cost': acc_cost / 60,
                     'nb_COFs_initialization': nb_COFs_initialization,
                     'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired.item(),
                     'elapsed_time (min)':  elapsed_time / 60
                    })

    with open('search_results/{}/sfbo_results/sfbo_results_run_{}.pkl'.format(normalization, i), 'wb') as file:
        pickle.dump(sfbo_res, file)

run #: 0
Iter 10/100: 2.5316478673813574
Iter 20/100: 1.5976805610155178
Iter 30/100: 1.5335137562067445
Iter 40/100: 1.5103225146173305
Iter 50/100: 1.4670837577792153
Iter 60/100: 1.46410267330956
BO iteration  3
	acquired COF  521
	y =  15.766064256252303
Iter 10/100: 2.2821451376626243
Iter 20/100: 1.5767457678144892
Iter 30/100: 1.5160547021724815
Iter 40/100: 1.4776026924306396
Iter 50/100: 1.4418954556648254
Iter 60/100: 1.43763657751167
BO iteration  4
	acquired COF  523
	y =  14.017515356923804
Iter 10/100: 2.118271469591821
Iter 20/100: 1.5406069636537973
Iter 30/100: 1.4658872536471157
Iter 40/100: 1.4001012223371432
Iter 50/100: 1.3578769891536706
Iter 60/100: 1.3454021759424304
BO iteration  5
	acquired COF  524
	y =  6.895659242359536
Iter 10/100: 2.0287369141547815
Iter 20/100: 1.541780078511085
Iter 30/100: 1.4659487823993385
Iter 40/100: 1.4034512338980265
Iter 50/100: 1.3717580593803287
Iter 60/100: 1.361105079632785
Iter 70/100: 1.3618015872844431
Iter 80/100: 1.3601

Iter 50/100: 1.170791985141159
BO iteration  42
	acquired COF  14
	y =  6.881047378855367
Iter 10/100: 1.5669156552054664
Iter 20/100: 1.4287022782510435
Iter 30/100: 1.3630345296394106
Iter 40/100: 1.2805961425355863
Iter 50/100: 1.2441196726974075
BO iteration  43
	acquired COF  515
	y =  6.237059454214897
Iter 10/100: 1.564915443037944
Iter 20/100: 1.4280230234141984
Iter 30/100: 1.3625088023252885
Iter 40/100: 1.2801281197458534
Iter 50/100: 1.2467900772603253
BO iteration  44
	acquired COF  149
	y =  4.870990027657603
Iter 10/100: 1.5626046190166525
Iter 20/100: 1.4269298032094793
Iter 30/100: 1.3612435080037733
Iter 40/100: 1.2792252398340007
Iter 50/100: 1.2486154006203505
BO iteration  45
	acquired COF  10
	y =  6.473829589979741
Iter 10/100: 1.5607194918637448
Iter 20/100: 1.4263514887445374
Iter 30/100: 1.3610154534330432
Iter 40/100: 1.280020433132811
Iter 50/100: 1.250579212370645
BO iteration  46
	acquired COF  282
	y =  2.9106750650702327
Iter 10/100: 1.5638979918518523
I

Iter 80/100: 1.2058886877964508
Iter 90/100: 1.2051956112787507
BO iteration  74
	acquired COF  451
	y =  2.7843374678631645
Iter 10/100: 1.5131796444574166
Iter 20/100: 1.3984405729030203
Iter 30/100: 1.3247586951550827
Iter 40/100: 1.264789376383358
Iter 50/100: 1.2283279626055488
Iter 60/100: 1.2039772202311516
Iter 70/100: 1.2031584488916516
Iter 80/100: 1.202458049973207
BO iteration  75
	acquired COF  450
	y =  3.890496901031045
Iter 10/100: 1.5119963318134042
Iter 20/100: 1.3976447635627007
Iter 30/100: 1.322634802820497
Iter 40/100: 1.2612240307847054
Iter 50/100: 1.2258649690069097
Iter 60/100: 1.2016294649772892
Iter 70/100: 1.2008071501791089
Iter 80/100: 1.2002387670218433
BO iteration  76
	acquired COF  36
	y =  2.8717389164600564
Iter 10/100: 1.5101388238235862
Iter 20/100: 1.3958654408563937
Iter 30/100: 1.319535858730366
Iter 40/100: 1.2576367532057053
Iter 50/100: 1.2225900640487841
Iter 60/100: 1.1984979330369878
Iter 70/100: 1.1975474451429378
Iter 80/100: 1.19713177

Iter 40/100: 1.1875067454684285
Iter 50/100: 1.1511676140764782
Iter 60/100: 1.1322599149604542
Iter 70/100: 1.1283887404776103
BO iteration  100
	acquired COF  60
	y =  3.8724205060110855
Iter 10/100: 1.481511956665914
Iter 20/100: 1.3686120048119117
Iter 30/100: 1.2692290220654674
Iter 40/100: 1.1843594730836995
Iter 50/100: 1.1485919239508402
Iter 60/100: 1.1298392497867786
Iter 70/100: 1.1260671832236708
BO iteration  101
	acquired COF  461
	y =  3.863653358562827
Iter 10/100: 1.4806311289499536
Iter 20/100: 1.3677256993579845
Iter 30/100: 1.2675800627635607
Iter 40/100: 1.1816430671356148
Iter 50/100: 1.1460391871211477
Iter 60/100: 1.1275555375694772
Iter 70/100: 1.12382527321168
Iter 80/100: 1.1236625872881618
BO iteration  102
	acquired COF  428
	y =  6.954408697894654
Iter 10/100: 1.479442580678323
Iter 20/100: 1.3665358509792416
Iter 30/100: 1.2653300466277586
Iter 40/100: 1.177027107937492
Iter 50/100: 1.1403317547976772
Iter 60/100: 1.1228496165800574
Iter 70/100: 1.1187620

Iter 60/100: 1.0546421968563777
Iter 70/100: 1.051575076666772
Iter 80/100: 1.0511113470709295
BO iteration  126
	acquired COF  477
	y =  3.1850961198447836
Iter 10/100: 1.4536660880657883
Iter 20/100: 1.3357244257507783
Iter 30/100: 1.2090938395725537
Iter 40/100: 1.08855566767103
Iter 50/100: 1.056805860011604
Iter 60/100: 1.0510148722875456
Iter 70/100: 1.0479239090044086
Iter 80/100: 1.0475224052381054
BO iteration  127
	acquired COF  359
	y =  2.616412984121879
Iter 10/100: 1.4525646048170393
Iter 20/100: 1.3344714022491844
Iter 30/100: 1.2068704713510987
Iter 40/100: 1.0847186113108322
Iter 50/100: 1.05307466497753
Iter 60/100: 1.0470568417267252
Iter 70/100: 1.044114959155093
Iter 80/100: 1.0437053136128238
BO iteration  128
	acquired COF  519
	y =  5.272175774230507
Iter 10/100: 1.4524147603642963
Iter 20/100: 1.3344217073063822
Iter 30/100: 1.2068105496732953
Iter 40/100: 1.0834003743365617
Iter 50/100: 1.0509643700877744
Iter 60/100: 1.0451838739572987
Iter 70/100: 1.04185621

...

Iter 30/100: 1.4831387411901549
Iter 40/100: 1.4323153237258108
Iter 50/100: 1.4094169633117268
Iter 60/100: 1.4061308844059333
Iter 70/100: 1.4018974758476117
Iter 80/100: 1.400322224599431
BO iteration  6
	acquired COF  583
	y =  2.903458451797101
Iter 10/100: 1.9719473416723203
Iter 20/100: 1.543591576165279
Iter 30/100: 1.4647936561908481
Iter 40/100: 1.4164442453225143
Iter 50/100: 1.3887549796846617
Iter 60/100: 1.3799536861988226
BO iteration  7
	acquired COF  475
	y =  3.1367526689994154
Iter 10/100: 1.91833299112707
Iter 20/100: 1.5358275458426212
Iter 30/100: 1.4565262408641644
Iter 40/100: 1.404711069439786
Iter 50/100: 1.3905128427363909
BO iteration  8
	acquired COF  335
	y =  5.584529614206016
Iter 10/100: 1.8744790886036964
Iter 20/100: 1.526640976711989
Iter 30/100: 1.4503933810678524
Iter 40/100: 1.396880287455372
Iter 50/100: 1.3851241576397675
Iter 60/100: 1.3776180419443225
BO iteration  9
	acquired COF  212
	y =  14.992345956579042
Iter 10/100: 1.85041767994821
Ite

Iter 30/100: 1.3864456382928083
Iter 40/100: 1.332474323590832
Iter 50/100: 1.2656809941469482
BO iteration  41
	acquired COF  88
	y =  3.561167963046627
Iter 10/100: 1.5754487204451413
Iter 20/100: 1.438204720393131
Iter 30/100: 1.3835270268501
Iter 40/100: 1.3283292024192161
Iter 50/100: 1.2625572714611681
BO iteration  42
	acquired COF  41
	y =  6.231560962484038
Iter 10/100: 1.5730844679450853
Iter 20/100: 1.437380500056505
Iter 30/100: 1.3826416238172454
Iter 40/100: 1.32688973115713
Iter 50/100: 1.2608509487316986
BO iteration  43
	acquired COF  71
	y =  6.71754508300835
Iter 10/100: 1.5716015457757455
Iter 20/100: 1.4371120085452447
Iter 30/100: 1.383381111934663
Iter 40/100: 1.3301569544500151
Iter 50/100: 1.2525259365883337
BO iteration  44
	acquired COF  482
	y =  3.2008641364107056
Iter 10/100: 1.5681455639893076
Iter 20/100: 1.4348527709236114
Iter 30/100: 1.3797759504861045
Iter 40/100: 1.3247383893114262
Iter 50/100: 1.259660569261645
BO iteration  45
	acquired COF  7
	y 

Iter 20/100: 1.3902846061453549
Iter 30/100: 1.3049101535235448
Iter 40/100: 1.2001338534823642
BO iteration  79
	acquired COF  370
	y =  3.8803155027900575
Iter 10/100: 1.5037025107360253
Iter 20/100: 1.3893964301548771
Iter 30/100: 1.3035696374991121
Iter 40/100: 1.1978024171801818
BO iteration  80
	acquired COF  44
	y =  3.465774886992771
Iter 10/100: 1.5026325904076252
Iter 20/100: 1.388519992560616
Iter 30/100: 1.3016936729280293
Iter 40/100: 1.19270375960179
BO iteration  81
	acquired COF  39
	y =  4.173430254080946
Iter 10/100: 1.5015205335697677
Iter 20/100: 1.3877086687600793
Iter 30/100: 1.3002277971369547
Iter 40/100: 1.1898009143310082
BO iteration  82
	acquired COF  516
	y =  5.711634635664843
Iter 10/100: 1.5005344742914402
Iter 20/100: 1.387155762436543
Iter 30/100: 1.299213093906544
Iter 40/100: 1.1877947405903253
BO iteration  83
	acquired COF  22
	y =  3.836424483128883
Iter 10/100: 1.4997105241950839
Iter 20/100: 1.3865886487218457
Iter 30/100: 1.297627621812672
Iter

Iter 30/100: 1.244338881242343
Iter 40/100: 1.1237113025404286
Iter 50/100: 1.05141558467667
Iter 60/100: 1.0453171646865758
Iter 70/100: 1.0421887091386097
Iter 80/100: 1.0413006660353594
Iter 90/100: 1.0410548396998835
Iter 100/100: 1.0408966059521803
BO iteration  109
	acquired COF  236
	y =  2.792100282751095
Iter 10/100: 1.470332317670368
Iter 20/100: 1.3552054948079277
Iter 30/100: 1.242138688680722
Iter 40/100: 1.120109255312901
Iter 50/100: 1.047183194007449
Iter 60/100: 1.0410475019432381
Iter 70/100: 1.0379347098134224
Iter 80/100: 1.037018699369941
Iter 90/100: 1.0367790805595987
Iter 100/100: 1.0366197127919319
BO iteration  110
	acquired COF  542
	y =  5.252390372733182
Iter 10/100: 1.4706048899656803
Iter 20/100: 1.3560850023391655
Iter 30/100: 1.24415779520578
Iter 40/100: 1.115549427171198
Iter 50/100: 1.0433762021366302
Iter 60/100: 1.0372344635965207
Iter 70/100: 1.0337893299759229
Iter 80/100: 1.0329037955124833
Iter 90/100: 1.03283268676023
Iter 100/100: 1.032630285

Iter 70/100: 1.0055814875291038
Iter 80/100: 1.0043487314490045
Iter 90/100: 1.0041674114135972
BO iteration  131
	acquired COF  209
	y =  4.498776005750538
Iter 10/100: 1.4572119955453122
Iter 20/100: 1.3422607673636584
Iter 30/100: 1.2167306421991793
Iter 40/100: 1.0784241977586582
Iter 50/100: 1.0082602948092747
Iter 60/100: 1.0063035333672588
Iter 70/100: 1.003499296571784
Iter 80/100: 1.0022220231623657
Iter 90/100: 1.0020389545014405
BO iteration  132
	acquired COF  16
	y =  3.8789713956254515
Iter 10/100: 1.456626783080712
Iter 20/100: 1.3416674277257126
Iter 30/100: 1.2154700013290205
Iter 40/100: 1.0756799506560062
Iter 50/100: 1.0056679905237247
Iter 60/100: 1.003819047715649
Iter 70/100: 1.0010739198146597
Iter 80/100: 0.999794173882172
Iter 90/100: 0.999618270178444
BO iteration  133
	acquired COF  38
	y =  2.7389170997427503
Iter 10/100: 1.4556973830730275
Iter 20/100: 1.3406004866307437
Iter 30/100: 1.2136338669668931
Iter 40/100: 1.0720174144936705
Iter 50/100: 1.0016016

# Random Search

In [13]:
nb_runs = 1000

rs_res = dict()
rs_res['ids_acquired']     = []
rs_res['cost_acquired']    = []
rs_res['accumulated_cost'] = []

for r in range(nb_runs):
    rs_ids_acquired     = np.random.choice(range(nb_COFs), replace=False, size=nb_iterations)
    rs_cost_acquired    = build_cost(rs_ids_acquired)
    rs_accumulated_cost = accumulated_cost(rs_cost_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)
    rs_res['accumulated_cost'].append(rs_accumulated_cost / 60)

In [14]:
def test_rs(rs_res):
    for a in np.random.choice(nb_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_iterations
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [15]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_runs)])
        assert np.size(y_maxes) == nb_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [16]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired'],
                             'accumulated_cost': rs_res['accumulated_cost']
                             })

with open('search_results/{}/random_search_results.pkl'.format(normalization), 'wb') as file:
    pickle.dump(random_search_results, file)